In [ ]:
For this project , i need to find authentic data myself to accuratly reprsent the rent health of the metropol. Kijiji seems like our best bet for many reason :
-Has the most listing for montreal
-French and english listing , so authenticly Montreal
-Consistent listing structure
We also do have some challenges with this choice:
-There is some antibot mesure so we need to scrape the data in a authorized manner.
-The data is gonna need some cleaning
-The price format sometimes differ (1,000$ or 1000$)


Let's start to import all the tools we're going to need

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

print("Libraries imported successfully")

Libraries imported successfully


Let's connect to Kijiji

In [6]:
url = "https://www.kijiji.ca/b-apartments-condos/ville-de-montreal/page-1/c37l1700281"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

response = requests.get(url, headers=headers)
print(f" Page loaded successfully Status: {response.status_code}")

 Page loaded successfully Status: 200


In [ ]:
Let's take a look at the page and save it

In [10]:
soup = BeautifulSoup(response.content, 'html.parser')

with open('kijiji_page.html', 'w', encoding='utf-8') as f:
    f.write(soup.prettify())
    print(" Saved page as 'kijiji_page.html'")





 Saved page as 'kijiji_page.html'


Findinng the listings (Sample size)

In [23]:

with open('kijiji_page.html', 'r', encoding='utf-8') as f:
    content = f.read()

print("Analyzing the HTML structure...")

#Count price mentions
price_matches = re.findall(r'\$\d+', content)
print(f"Price mentions found: {len(price_matches)}")

#Check for common listing indicators
listing_indicators = ['search-item', 'listing', 'result', 'item-container', 'data-testid']
found_indicators = []

for indicator in listing_indicators:
    if indicator in content.lower():
        found_indicators.append(indicator)

if found_indicators:
    print(f"Listing indicators found: {found_indicators}")
else:
    print("No common listing indicators found")

#Check for JavaScript data loading
if 'window.__data' in content or 'JSON.parse' in content:
    print("⚠️  JavaScript data loading detected")
else:
    print("No heavy JavaScript detected")

#Show HTML sample if prices exist
if price_matches:
    print("\n HTML sample around first price:")
    price_index = content.find(price_matches[0])
    sample = content[max(0, price_index-200):price_index+200]
    print(sample)
else:
    print("No prices found in HTML")

Analyzing the HTML structure...
Price mentions found: 80
Listing indicators found: ['listing', 'result', 'data-testid']
No heavy JavaScript detected

 HTML sample around first price:
ttps://schema.org","@type":"SingleFamilyResidence","name":"Métro Jarry - Beau 3 1/2 - Près de tout!","description":"Beau 3 1/2 rénové - Près de tous les services! 1400 Jarry Est, Montréal À partir de $1 095 /moisCaractéristiques : Immeuble en béton, bien isolé Cuisinière + Réfrigérateur inclus Ascenseur Balcon ...","image":"https://media.kijiji.ca/api/v1/ca-prod-dealer-ads/images/ae/aeb7d3f2-c271-


This section analyzes the Kijiji webpage to understand how it's built, so we can tell our scraper exactly where to find the apartment listing data

In [17]:
# Find the exact data-testid values from the saved HTML
with open('kijiji_page.html', 'r', encoding='utf-8') as f:
    content = f.read()

soup = BeautifulSoup(content, 'html.parser')

print("Finding data-testid values...")

# Find all elements with data-testid
data_testids = soup.find_all(attrs={"data-testid": True})
print(f"Total elements with data-testid: {len(data_testids)}")

# Collect unique data-testid values
testids = set()
for element in data_testids:
    testid = element.get('data-testid')
    if testid:
        testids.add(testid)

print("All data-testid values found:")
for testid in sorted(testids):
    print(f"  {testid}")

# Identify listing-related testids
listing_testids = [tid for tid in testids if any(word in tid for word in ['listing', 'card', 'item', 'result'])]
print(f"Listing-related testids: {listing_testids}")

# Examine a sample listing structure
if listing_testids:
    sample_listing = soup.find(attrs={"data-testid": listing_testids[0]})
    if sample_listing:
        print("Sample listing structure:")
        print(sample_listing.prettify()[:1500])

Finding data-testid values...
Total elements with data-testid: 838
All data-testid values found:
  accordion-wrapper
  additional-options
  additional-options-list
  adsense-container
  adsense-ins
  all-filters-button
  breadcrumb-link-0
  breadcrumb-link-1
  breadcrumb-link-2
  breadcrumb-link-3
  breadcrumbs-nav
  dot-divider
  global-header-desktop-button
  global-header-logo
  global-header-mobile-button-footer
  global-header-search-bar
  global-header-tools
  global-search-bar
  gpt-leaderboard-top
  gpt-leaderboard-top-in-view
  gpt-skyscraper
  header-button-submit
  header-link-post-ad
  header-register
  header-sign-in
  language-toggle
  listing-badge-featured
  listing-card
  listing-card-image
  listing-card-image-container
  listing-card-image-container-highlight
  listing-card-list-item-0
  listing-card-list-item-1
  listing-card-list-item-10
  listing-card-list-item-11
  listing-card-list-item-12
  listing-card-list-item-13
  listing-card-list-item-14
  listing-card-li

Based on the analysis, here's the targeted scraper that will work with Kijiji's structure:

In [18]:
# Section 3: Kijiji Apartment Scraper Implementation

class KijijiMontrealScraper:
    def __init__(self):
        self.base_url = "https://www.kijiji.ca/b-apartments-condos/ville-de-montreal/page-{}/c37l1700281"
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
    
    def extract_price(self, text):
        """Extract price number from text"""
        if text:
            numbers = re.findall(r'\d+', str(text))
            if numbers:
                return int(''.join(numbers))
        return None
    
    def extract_bedrooms(self, text):
        """Extract bedroom count from text"""
        text = str(text).lower()
        patterns = [
            r'(\d+)\s*bedroom',
            r'(\d+)\s*br',
            r'(\d+)\s*chambre',
            r'studio'
        ]
        
        for pattern in patterns:
            match = re.search(pattern, text)
            if match:
                if pattern == 'studio':
                    return 0
                return int(match.group(1))
        return None
    
    def scrape_page(self, page_num):
        """Scrape one page of Kijiji listings"""
        url = self.base_url.format(page_num)
        print(f"Scraping page {page_num}")
        
        try:
            response = requests.get(url, headers=self.headers)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Find all listing cards using the correct data-testid
            listing_cards = soup.find_all(attrs={"data-testid": "listing-card"})
            print(f"Found {len(listing_cards)} listing cards")
            
            page_data = []
            for card in listing_cards:
                listing_data = self.extract_listing_data(card)
                if listing_data and listing_data.get('price'):
                    page_data.append(listing_data)
            
            return page_data
            
        except Exception as e:
            print(f"Error scraping page {page_num}: {e}")
            return []
    
    def extract_listing_data(self, card):
        """Extract data from a single listing card"""
        try:
            # Extract title using the correct data-testid
            title_elem = card.find(attrs={"data-testid": "listing-title"})
            title = title_elem.text.strip() if title_elem else "No title"
            
            # Extract price using the correct data-testid
            price_elem = card.find(attrs={"data-testid": "listing-price"})
            price_text = price_elem.text.strip() if price_elem else ""
            price = self.extract_price(price_text)
            
            # Extract location using the correct data-testid
            location_elem = card.find(attrs={"data-testid": "listing-location"})
            location = location_elem.text.strip() if location_elem else "Montreal"
            
            # Extract description using the correct data-testid
            desc_elem = card.find(attrs={"data-testid": "listing-description"})
            description = desc_elem.text.strip() if desc_elem else ""
            
            # Extract bedrooms from title and description
            bedrooms = self.extract_bedrooms(title + " " + description)
            
            return {
                'title': title,
                'price': price,
                'bedrooms': bedrooms,
                'location': location,
                'description': description[:200],  # First 200 characters
                'scraped_at': pd.Timestamp.now()
            }
            
        except Exception as e:
            print(f"Error extracting listing data: {e}")
            return None
    
    def scrape_multiple_pages(self, num_pages=5):
        """Scrape multiple pages with delays"""
        all_data = []
        
        for page in range(1, num_pages + 1):
            page_data = self.scrape_page(page)
            all_data.extend(page_data)
            
            # Respectful delay between requests
            time.sleep(2)
            
            print(f"Page {page}: collected {len(page_data)} listings")
        
        return all_data
    
    def save_to_dataframe(self, data):
        """Convert scraped data to pandas DataFrame"""
        df = pd.DataFrame(data)
        
        # Basic data validation
        initial_count = len(df)
        df = df.dropna(subset=['price'])  # Remove listings without price
        df = df[df['price'] > 300]  # Remove unrealistic prices
        
        print(f"Data cleaning: {initial_count} -> {len(df)} valid listings")
        return df

# Test the scraper
print("Starting Kijiji scraper test...")
scraper = KijijiMontrealScraper()

# Scrape 3 pages as a test
test_data = scraper.scrape_multiple_pages(num_pages=3)
df = scraper.save_to_dataframe(test_data)

print(f"Scraping completed. Total listings: {len(df)}")
print("\nFirst 5 listings:")
print(df.head())

# Save to CSV for the next notebook
df.to_csv('kijiji_montreal_rentals.csv', index=False)
print("Data saved to kijiji_montreal_rentals.csv")

Starting Kijiji scraper test...
Scraping page 1
Found 46 listing cards
Page 1: collected 45 listings
Scraping page 2
Found 46 listing cards
Page 2: collected 46 listings
Scraping page 3
Found 46 listing cards
Page 3: collected 46 listings
Data cleaning: 137 -> 137 valid listings
Scraping completed. Total listings: 137

First 5 listings:
                                               title   price  bedrooms  \
0  Lasalle, Grand 4½, NEUF, Tout inclus, Parking ...  150000       NaN   
1  Cession de bail - 4 1/2 St-Henri – 1975$/mois ...  197500       NaN   
2  Super 4-1/2 Avenue Kent - Darlington/Van Horne...  150000       NaN   
3  Grands 4 1/2 NEUFS Métro Pie-IX (1er mois GRAT...  197500       NaN   
4  renovated studio apartment near blvd Saint-Lar...  109500       0.0   

                          location  \
0          Cavelier Park, Montréal   
1            Saint-Henry, Montreal   
2                         Montreal   
3  Hochelaga-Maisonneuve, Montréal   
4                         

In [ ]:
This concludes the data collection phase; the cleaned dataset has been saved to 'kijiji_montreal_rentals.csv' for analysis in the next notebook.